In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
!killall python

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit


def loader(train=r"test_data_hava_query_title.csv"):##################就是很简单的实现，先read_csv,然后填充缺失值为每列均值
    all_data=pd.read_csv(train)
    all_data=all_data[['query','title','title_count','title_ctr','label']]
    mean_val = all_data['title_count'].mean()
    all_data['title_count'].fillna(mean_val, inplace=True)
    mean_val = all_data['title_ctr'].mean()
    all_data['title_ctr'].fillna(mean_val, inplace=True)
    data_split=StratifiedShuffleSplit(n_splits=2,test_size=0.02,random_state=1)
    train_index,vaild_index=data_split.split(all_data['label'],all_data['label']).__next__()
    train_data=all_data.iloc[train_index]
    vaild_data=all_data.iloc[vaild_index]
    return train_data,vaild_data

############下面的函数返回一个迭代器，迭代一次产生一个batch
def batch_iter(data_x,data_y, batch_size, num_epochs, shuffle=True):#这个就是产生batch的，可以直接用
    """
    Generates a batch iterator for a dataset.
    """
    
    data=list(zip(data_x,data_y))
    data=np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:#如果为true 则代表允许随机取数据
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]#随机取数据
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]#而不用一个个加到列表里了。这是一个batch，嗯哼
            #取到的维度为（batchsize,2） y为(batchsize,1) 里面存的是序号



















import tensorflow as tf

import pandas as pd
import numpy as np
from gensim.models import Word2Vec
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)
def weight_variable(shape,name):
    with tf.variable_scope('model',reuse=False):
        initial = tf.get_variable(name , shape , initializer = tf.contrib.layers.variance_scaling_initializer())
    return initial
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding = 'VALID')
    # tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
    # x(input)  : [batch, in_height, in_width, in_channels]
    # W(filter) : [filter_height, filter_width, in_channels, out_channels]
    # strides   : The stride of the sliding window for each dimension of input.
    #             For the most common case of the same horizontal and vertices strides, strides = [1, stride, 
def max_pool_2x2(x,ksize_input):
    return tf.nn.max_pool(x, ksize =ksize_input,
                          strides = [1, 1, 1, 1], padding = 'VALID')


def batchnorm( Ylogits, offset, convolutional=False):
        """batchnormalization.
        Args:
                Ylogits: 1D向量或者是3D的卷积结果。
                num_updates: 迭代的global_step
                offset：表示beta，全局均值；在 RELU 激活中一般初始化为 0.1。
                scale：表示lambda，全局方差；在 sigmoid 激活中需要，这 RELU 激活中作用不大。
                m: 表示batch均值；v:表示batch方差。
                bnepsilon：一个很小的浮点数，防止除以 0.
        Returns:
                Ybn: 和 Ylogits 的维度一样，就是经过 Batch Normalization 处理的结果。
                update_moving_everages：更新mean和variance，主要是给最后的 test 使用。
        """
        
        exp_moving_avg = tf.train.ExponentialMovingAverage(0.999,
                                                                global_step)  # adding the iteration prevents from averaging across non-existing iterations
        bnepsilon = 1e-5
        if convolutional:
                mean, variance = tf.nn.moments(Ylogits, [0, 1, 2])
        else:
                mean, variance = tf.nn.moments(Ylogits, [0])
        update_moving_everages = exp_moving_avg.apply([mean, variance])
        m = tf.cond(tst, lambda: exp_moving_avg.average(mean), lambda: mean)
        v = tf.cond(tst, lambda: exp_moving_avg.average(variance), lambda: variance)
        Ybn = tf.nn.batch_normalization(Ylogits, m, v, offset, None, bnepsilon)
        return Ybn, update_moving_everages


global_step=tf.Variable(0,name='global_step',trainable=False)
def ln_model(data) :
    ###############输入是两维度，1、ctr 2、count
    #得到数据的长度，即每个样例有多少个输入
    len = 2#######################################这里需要改成你特征的维度数，即你特征的个数

    #随机初始化参数
    tf.set_random_seed(1)
    with tf.variable_scope('ln_model',reuse=False):
        W1 = tf.get_variable("W1" , [len , 8] , initializer = tf.contrib.layers.variance_scaling_initializer())
        b1 = bias_variable([8])
        W2 = tf.get_variable("W2" , [8 , 1] , initializer = tf.contrib.layers.variance_scaling_initializer())
        b2 = bias_variable([1])

    #前向传播：
    Z1 = tf.nn.bias_add(tf.matmul(data , W1) , b1)
    Z2 = tf.nn.bias_add(tf.matmul(Z1 , W2) , b2)
    A = tf.nn.relu(Z2)


    # 输出每个样本经过计算的值
    output = tf.reshape(A, [-1, 1])
    return output################出来一个[1,1]


def get_w2v_array(word_list,max_len):##################初始化一个固定长度的全零矩阵，超过这个长度就截断，
    array = np.zeros((max_len, embedding_size))
    if len(word_list)<=max_len:
        for i in range(len(word_list)):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
    else:
        for i in range(max_len):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
    return array

###########################
query_len=6###################就是query和title 需要取一个固定长度。
title_len=19
embedding_size=120

filter_sizes = [2, 3, 4, 5]

num_filters=256
fc_hidden_size=1024

lr=0.006
decay_step=5000

update_emas=[]

input_x_query=tf.placeholder(tf.float32,[None,query_len,embedding_size])#h r
input_x_title=tf.placeholder(tf.float32,[None,title_len,embedding_size])# r
input_ctr_fea=tf.placeholder(tf.float32,[None,2])# ctr特征
input_y=tf.placeholder(tf.float32,[None,1])#就是 标签
keep_prob = tf.placeholder(tf.float32, [])
tst = tf.placeholder(tf.bool)

# W_h = tf.Variable(
#     tf.random_uniform([vocab_size_e, embedding_size], -1.0, 1.0),
#     )#[None，1，10]
# W_r = tf.Variable(
#     tf.random_uniform([vocab_size_r, embedding_size], -1.0, 1.0),
#     )#[None，1，10] h和r
#########################################词向量部分 会替换成预训练词向量矩阵


embedded_x_query = tf.expand_dims(input_x_query, -1)#-1代表最后一维

embedded_x_title = tf.expand_dims(input_x_title, -1)#-1代表最后一维
#####[None,title_len,embedding_size,1]##最后一维是通道
##############################################################这上面分别换成query和title
#过六个大小为[1,embedding_size,1,3]的卷积核
pooled_all=[]


for i,filter_size in enumerate(filter_sizes):
    with tf.name_scope("conv1-query-%s" % i):
        filter_shape = [filter_size, embedding_size, 1, num_filters]
        W=weight_variable(filter_shape,name="w_query"+str(i))
        b = bias_variable([num_filters])
        print(embedded_x_query)
        print(W,"**********")
        conv=conv2d(embedded_x_query,W)#→[none,1,1,num_filters]
        conv_bn, update_ema = batchnorm(conv, b, convolutional=True)  # 在激活层前面加 BN
        update_emas.append(update_ema)
        query_relu = tf.nn.relu(conv_bn, name="relu")
        ksize=[1,query_len - filter_size + 1,1,1]
        pooled=max_pool_2x2(query_relu,ksize)
        pooled_all.append(pooled)
query_pool = tf.concat(pooled_all,3)#→[none,1,1,num_filters*len(filter_sizes)]
query_pool_flat = tf.reshape(query_pool, [-1, len(filter_sizes)*num_filters])
pooled_all=[]


for i,filter_size in enumerate(filter_sizes):
    with tf.name_scope("conv1-title-%s" % i):
        filter_shape = [filter_size, embedding_size, 1, num_filters]
        W=weight_variable(filter_shape,name="w_title"+str(i))
        b = bias_variable([num_filters])
        conv=conv2d(embedded_x_title,W)
        conv_bn, update_ema = batchnorm(conv, b, convolutional=True)
        update_emas.append(update_ema)
        title_relu = tf.nn.relu(conv_bn, name="relu")
        ksize=[1,title_len - filter_size + 1,1,1]
        pooled=max_pool_2x2(title_relu,ksize)
        pooled_all.append(pooled)
title_pool=tf.concat(pooled_all,axis=3)
title_pool_flat = tf.reshape(title_pool, [-1, len(filter_sizes)*num_filters])


output = tf.concat([query_pool_flat, title_pool_flat], axis=1)
W_fc = weight_variable([len(filter_sizes)*num_filters * 2, fc_hidden_size],name="fc")
h_fc = tf.matmul(output, W_fc, name='h_fc')
beta_fc = tf.Variable(tf.constant(0.1, tf.float32, shape=[fc_hidden_size], name="beta_fc"))
fc_bn, update_ema_fc = batchnorm(h_fc, beta_fc, convolutional=False)
update_emas.append(update_ema_fc)
fc_bn_relu = tf.nn.relu(fc_bn, name="relu")
fc_bn_drop = tf.nn.dropout(fc_bn_relu, keep_prob)


with tf.variable_scope('out_layer'):
    W_out = weight_variable([fc_hidden_size, 1],name="output")
    b_out = bias_variable([1])
    _y_pred_nn = tf.nn.xw_plus_b(fc_bn_drop, W_out, b_out, name='y_pred')  # 每个类别的分数 scores
    _y_pred_ln = ln_model(input_ctr_fea)###########data还得写
    _y_pred_nn_ln=tf.add(_y_pred_nn , _y_pred_ln)
    central_bias = tf.Variable([np.random.randn()], name="central_bias")###初始化还得想想！！！！
    _y_pred = tf.multiply(_y_pred_nn_ln, central_bias)

with tf.name_scope('loss'):

    loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(logits=_y_pred, labels=input_y))
    
    learning_rate = tf.train.exponential_decay(lr, global_step, decay_step,
        0.8, staircase=True)
    optimizer=tf.train.AdamOptimizer(learning_rate)
    grads_and_vars=optimizer.compute_gradients(loss)
    train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)

    ###############################输入的名字 _y_inputs也得换 之类的！！！

######剩下的，还需要看截断维度！！！！！！！！



sess = tf.Session() #启动创建的模型
sess.run(tf.initialize_all_variables())




train_batches,vaild_data=loader()
train_batches=np.array(train_batches)
train_batches_y=train_batches[:,4]
train_batches_x=np.delete(train_batches, 4, axis=1)######这里是除了label以外的维数

batches=batch_iter(train_batches_x,train_batches_y,1024,4)

vaild_batches=np.array(vaild_data)
vaild_batches_y=vaild_batches[:,4]
vaild_batches_x=np.delete(vaild_batches, 4, axis=1)
vaild_data=batch_iter(vaild_batches_x,vaild_batches_y,5000,100)

print("train starting")
w2v_model = Word2Vec.load('w2v_model/w2v_all_data_model.txt')
word_wv= w2v_model.wv


per_train_loss_avg_list=[]
per_vaild_loss_avg_list=[]
for batch in batches:
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],title_len),axis=1,arr=batch_x)

    ######x[0]暂时代表query的位置 x[1]表示title的位置
    batch_x_ctr_fea=batch_x[:,2:]######这里需要到时候调整，按label的位置和特征位置
    
    feed_dict={
            input_x_query:batch_x_query,
            input_x_title:batch_x_title,
            input_ctr_fea:batch_x_ctr_fea,
            input_y:batch_y,#这个就是label
            tst:False,
            keep_prob:0.8
    }
    
    _,step,loss_out=sess.run([train_op,global_step,loss],feed_dict)
    print("step {},loss:{:g}".format(step,loss_out))
    per_train_loss_avg_list.append(loss_out)
    if step%(int(5900000/1024))==0:
        print("batch ended")
    if step%50==0:
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    
        #####[None,title_len,embedding_size]
        #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
        #query   title   title_count         title_ctr      label
        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],title_len),axis=1,arr=vaild_x)
    
        ######x[0]暂时代表query的位置 x[1]表示title的位置
        vaild_x_ctr_fea=vaild_x[:,2:]######这里需要到时候调整，按label的位置和特征位置
        feed_dict={
                input_x_query:vaild_x_query,
                input_x_title:vaild_x_title,
                input_ctr_fea:vaild_x_ctr_fea,
                input_y:vaild_y,#这个就是label
                tst:False,
                keep_prob:1
        }
        step,loss_out=sess.run([global_step,loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%500==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")
        saver = tf.train.Saver(max_to_keep = 3)
        saver = saver.save(sess, "tf_model/model.ckpt", global_step=step)

    



# 这个是如果上面代码跑着跑着看不到输出了，你可以直接执行下面的代码，接着上面的运行

In [ ]:


for batch in batches:
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],title_len),axis=1,arr=batch_x)

    ######x[0]暂时代表query的位置 x[1]表示title的位置
    batch_x_ctr_fea=batch_x[:,2:]######这里需要到时候调整，按label的位置和特征位置
    
    feed_dict={
            input_x_query:batch_x_query,
            input_x_title:batch_x_title,
            input_ctr_fea:batch_x_ctr_fea,
            input_y:batch_y,#这个就是label
            tst:False,
            keep_prob:0.8
    }
    
    _,step,loss_out=sess.run([train_op,global_step,loss],feed_dict)
    print("step {},loss:{:g}".format(step,loss_out))
    per_train_loss_avg_list.append(loss_out)
    if step%(int(10000000/1024))==0:
        print("batch ended")
    if step%50==0:
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    
        #####[None,title_len,embedding_size]
        #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
        #query   title   title_count         title_ctr      label
        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],title_len),axis=1,arr=vaild_x)
    
        ######x[0]暂时代表query的位置 x[1]表示title的位置
        vaild_x_ctr_fea=vaild_x[:,2:]######这里需要到时候调整，按label的位置和特征位置
        feed_dict={
                input_x_query:vaild_x_query,
                input_x_title:vaild_x_title,
                input_ctr_fea:vaild_x_ctr_fea,
                input_y:vaild_y,#这个就是label
                tst:False,
                keep_prob:1
        }
        step,loss_out=sess.run([global_step,loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%500==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")
        saver = tf.train.Saver(max_to_keep = 3)
        saver = saver.save(sess, "tf_model_1000W/model-1000W.ckpt", global_step=step)

    